# AI-Powered Assistant for Intelligent Legal Review

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook simulates an AI assistant for legal professionals. It demonstrates two key capabilities:
1.  **Contract Analysis:** Automatically scanning a contract to flag potentially risky or non-standard clauses.
2.  **Case Law Summarization:** Using an extractive summarization algorithm to distill a long legal document to its key sentences.

### 1. Setup and Library Imports

In [ ]:
!pip install -q spacy scikit-learn pandas
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from collections import Counter

### 2. Tool 1: The Contract Risk Analyzer

This tool works by searching for specific keywords and phrases that are associated with common types of contractual risk. We first define our 'knowledge base' of these risky terms.

In [ ]:
RISK_KNOWLEDGE_BASE = {
    'UNLIMITED_LIABILITY': ['unlimited liability', 'liable without limitation', 'no cap on liability', 'full liability'],
    'NON_MUTUAL_INDEMNIFICATION': ['indemnify and hold harmless', 'defend and indemnify'],
    'AUTOMATIC_RENEWAL': ['automatically renew', 'continue automatically', 'unless terminated prior'],
    'ARBITRATION': ['binding arbitration', 'disputes shall be settled by arbitration']
}

# Sample Contract with embedded risky clauses
sample_contract = """
This Services Agreement is made between a Client and a Provider.
1. Services. The Provider shall perform services as described in Exhibit A.
2. Term. This agreement shall last for one year and will automatically renew for successive one-year terms unless terminated prior to 60 days before the renewal date.
3. Liability. Except for instances of gross negligence, the Provider's liability shall be limited to the fees paid in the preceding six months. The Client, however, agrees to full liability for any misuse of the services.
4. Indemnification. The Client agrees to indemnify and hold harmless the Provider against all claims, damages, and expenses.
5. Dispute Resolution. Any disputes shall be settled by arbitration in the state of New York.
"""

class ContractReviewAgent:
    def __init__(self, risk_kb):
        self.risk_kb = risk_kb
    
    def review(self, contract_text):
        print("--- Running Contract Risk Analysis ---")
        findings = []
        # Split contract into clauses (simplified as sentences)
        clauses = [s.strip() for s in contract_text.split('.') if s.strip()]
        
        for clause in clauses:
            for risk_category, keywords in self.risk_kb.items():
                for keyword in keywords:
                    if re.search(f'\\b{keyword}\\b', clause, re.IGNORECASE):
                        findings.append({'risk_type': risk_category, 'clause': clause})
                        break # Move to next clause once a risk is found
        return findings

# --- DEMONSTRATION of Tool 1 ---
contract_agent = ContractReviewAgent(RISK_KNOWLEDGE_BASE)
contract_findings = contract_agent.review(sample_contract)

if contract_findings:
    print(f"\nFound {len(contract_findings)} potential risks:\n")
    for finding in contract_findings:
        print(f"- RISK TYPE: {finding['risk_type']}")
        print(f"  CLAUSE: "...{finding['clause']}."\n")
else:
    print("No specific risks from the knowledge base were found.")

### 3. Tool 2: The Case Law Summarizer

This tool uses an **extractive summarization** technique. It scores each sentence in a document based on its TF-IDF value (a measure of how important its words are to the document) and extracts the highest-scoring sentences to form a summary.

In [ ]:
# Sample Case Law Text (long and dense)
case_law_text = """
The case of Fictional Corp v. Another Corp (2025) revolved around the interpretation of the 'force majeure' clause in a supply agreement following an unprecedented global logistical shutdown. The plaintiff, Fictional Corp, argued that the shutdown was a foreseeable event and did not excuse the defendant's failure to deliver critical components. The core legal question was whether the definition of 'Act of God' could be extended to include systemic, man-made supply chain disruptions. The court's ruling established a new precedent for such cases. The defendant, Another Corp, presented evidence that the disruption was of a scale so vast and unpredictable that it could not have been reasonably mitigated. They argued that this satisfied the requirements of the force majeure clause, thereby releasing them from their contractual obligations without penalty. The final judgment hinged on the specific wording of the contract, which failed to explicitly exclude global pandemics or logistical failures from the force majeure definition. The court ultimately found in favor of the defendant, Another Corp, highlighting the critical importance of precise and forward-looking language in contract drafting. This ruling has had a significant impact on how commercial contracts are written, with many legal experts now advising clients to explicitly define the scope of such clauses to avoid future ambiguity. The precedent underscores the dynamic nature of contract law in the face of modern global challenges.
"""

class SummarizationAgent:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.vectorizer = TfidfVectorizer(stop_words='english')
        
    def summarize(self, document_text, num_sentences=3):
        print("\n--- Running Case Law Summarization ---")
        # 1. Split into sentences
        doc = self.nlp(document_text)
        sentences = [sent.text for sent in doc.sents]
        
        # 2. Calculate TF-IDF scores for the whole document
        tfidf_matrix = self.vectorizer.fit_transform(sentences)
        
        # 3. Score each sentence by summing the TF-IDF scores of its words
        sentence_scores = tfidf_matrix.sum(axis=1)
        
        # 4. Get the indices of the top N sentences
        top_sentence_indices = np.argsort(sentence_scores.A.flatten())[-num_sentences:][::-1]
        top_sentence_indices.sort() # Sort to maintain original order
        
        # 5. Build the summary
        summary = ' '.join([sentences[i] for i in top_sentence_indices])
        return summary, len(doc.sents), num_sentences

# --- DEMONSTRATION of Tool 2 ---
summarizer_agent = SummarizationAgent()
summary, original_len, summary_len = summarizer_agent.summarize(case_law_text)

print(f"\nOriginal document length: {original_len} sentences.")
print(f"Summary length: {summary_len} sentences.\n")
print("--- GENERATED SUMMARY ---")
print(summary)